In [20]:
# CAP 5630 HW3
# Josh E.

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os


# assumes csv's are in the same directory
train_data = pd.read_csv(os.getcwd() + "\\train.csv")

# data preprocessing

first_class = train_data.loc[train_data.Pclass == 1]["Survived"]
#print("rate:", sum(first_class)/len(first_class))
# Survival rate of first-class: 0.6296296296296297

second_class = train_data.loc[train_data.Pclass == 2]["Survived"]
#print("rate:", sum(second_class)/len(second_class))
# Survival rate of second-class: 0.47282608695652173

third_class = train_data.loc[train_data.Pclass == 3]["Survived"]
#print("rate:", sum(third_class)/len(third_class))
# Survival rate of third-class: 0.24236252545824846

women = train_data.loc[train_data.Sex == 'female']["Survived"]
#print("rate:", sum(women)/len(women))
# Survival rate of women: 0.7420382165605095

men = train_data.loc[train_data.Sex == 'male']["Survived"]
#print("rate:", sum(men)/len(men))
# survival rate of men: 0.18890814558058924

all_ages = train_data[["Age", "Survived"]]
Age_Survived = [0] * (int(all_ages["Age"].max()) + 1) # indexes are age
Age_Died = [0] * (int(all_ages["Age"].max()) + 1)
for i in range(len(all_ages["Age"])):
    
    # remove people without age from analysis (177 missing, 714 valid)
    passenger_age = all_ages["Age"][i]
    if math.isnan(passenger_age):
        continue
    passenger_age = int(passenger_age)
    if all_ages["Survived"][passenger_age] == 1:
        Age_Survived[passenger_age] +=1
    else:
        Age_Died[passenger_age] +=1

age_range = 10
Age_Survival_Rate = [0] * (int(all_ages["Age"].max() / age_range) + 1) # For each age, list the survival rate within their age range

for i in range(len(Age_Survival_Rate)):
    #total_people = 0;
    total_survived = 0;
    for j in range(age_range):
        if i*age_range + j >= len(Age_Survived):
            break;
        #total_people = total_people + Age_Survived[i*age_range + j] + Age_Died[i*age_range + j]
        total_survived = total_survived + Age_Survived[i*age_range + j]
    Age_Survival_Rate[i] = total_survived / (sum(Age_Survived) + sum(Age_Died)) 
    
#print(Age_Survival_Rate)
#xdata = np.arange(start = 0, step = age_range, stop = len(Age_Survival_Rate) * age_range)
#plt.plot(xdata, Age_Survival_Rate)
#plt.show()
# age shows some sign of survivorship. A passenger is more likely to survive if he/she is of the ages 15 - 30. More generally, a passenger is more likely to survive if he/she is younger.

SibSp = train_data[["SibSp", "Survived"]]
SibSp_Survived = [0] * (int(SibSp["SibSp"].max()) + 1) # indexes are number of SibSp
SibSp_Died = [0] * (int(SibSp["SibSp"].max()) + 1)
for i in range(len(SibSp["SibSp"])):
    passenger_SibSp = SibSp["SibSp"][i]
    passenger_SibSp = int(passenger_SibSp)
    if SibSp["Survived"][passenger_SibSp] == 1:
        SibSp_Survived[passenger_SibSp] +=1
    else:
        SibSp_Died[passenger_SibSp] +=1

SibSp_Range = 2
SibSp_Survival_Rate = [0] * (int(SibSp["SibSp"].max() / SibSp_Range) + 1) # For each number of siblings, list the survival rate within their range


for i in range(len(SibSp_Survival_Rate)):
    total_survived = 0;
    for j in range(SibSp_Range):
        if i*SibSp_Range + j >= len(SibSp_Survived):
            break;
        total_survived = total_survived + SibSp_Survived[i*SibSp_Range + j]
    SibSp_Survival_Rate[i] = total_survived / (sum(SibSp_Survived) + sum(SibSp_Died)) 
    
#print(SibSp_Survival_Rate)
#xdata = np.arange(start = 0, step = SibSp_Range, stop = len(SibSp_Survival_Rate) * SibSp_Range)
#plt.plot(xdata, SibSp_Survival_Rate)
#plt.show()
# plot of number of SibSp vs Survival Rate indicates negative trend, but must consider that families with more sibling are rarer, which can skew data


Parch = train_data[["Parch", "Survived"]]
Parch_Survived = [0] * (int(Parch["Parch"].max()) + 1)
Parch_Died = [0] * (int(Parch["Parch"].max()) + 1)
for i in range(len(Parch["Parch"])):
    passenger_Parch = Parch["Parch"][i]
    passenger_Parch = int(passenger_Parch)
    if Parch["Survived"][passenger_Parch] == 1:
        Parch_Survived[passenger_Parch] +=1
    else:
        Parch_Died[passenger_Parch] +=1

Parch_Survival_Rate = [0] * (int(Parch["Parch"].max()) + 1)
for i in range(len(Parch_Survival_Rate)):
    Parch_Survival_Rate[i] = Parch_Survived[i] / (sum(Parch_Survived) + sum(Parch_Died)) 

#xdata = np.arange(start = 0, step = 1, stop = len(Parch_Survival_Rate))
#plt.scatter(xdata, Parch_Survival_Rate)
#plt.show()
# Shows the father and daughters are most likely to survive

from sklearn.model_selection import train_test_split

drop_columns = ["Survived", "PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Fare"]


X_train, X_test, y_train, y_test = train_test_split(train_data.drop(drop_columns, axis=1, inplace=False), train_data[["Survived"]], test_size=0.33)
X_train["Age"] = X_train["Age"].fillna(X_train["Age"].mean())
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].mean())
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

from sklearn.svm import SVC

linear_support_vector_machine = SVC(kernel='linear')
linear_support_vector_machine.fit(X_train, y_train.values.ravel())

from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(linear_support_vector_machine, X_train, y_train.values.ravel(), cv=5)
print("Average Classification Accuracy for Linear SVM: %f" % cv_score.mean())

quad_support_vector_machine = SVC(kernel='poly', degree=2, gamma='auto')
quad_support_vector_machine.fit(X_train, y_train.values.ravel())

cv_score = cross_val_score(quad_support_vector_machine, X_train, y_train.values.ravel(), cv=5)
print("Average Classification Accuracy for quadratic SVM: %f" % cv_score.mean())

rbf_support_vector_machine = SVC(kernel='rbf', gamma='auto')
rbf_support_vector_machine.fit(X_train, y_train.values.ravel())

cv_score = cross_val_score(rbf_support_vector_machine, X_train, y_train.values.ravel(), cv=5)
print("Average Classification Accuracy for RBF SVM: %f" % cv_score.mean())




Average Classification Accuracy for Linear SVM: 0.798613
Average Classification Accuracy for quadratic SVM: 0.832227
Average Classification Accuracy for RBF SVM: 0.813768
